In [ ]:
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
	left, right = list(), list()
	for row in dataset:
		if row[index] < value:
			left.append(row)
		else:
			right.append(row)
	return left, right

# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
	# count all samples at split point
    #n_instances=numero totale di unità statistiche (ottenuto sommando il numero di elementi
    #di entrambi gli insiemi)
	n_instances = float(sum([len(group) for group in groups]))
	# sum weighted Gini index for each group
	gini = 0.0
	for group in groups:
        #size è il numero di unità statistiche nel gruppo che stiamo considerando
		size = float(len(group))
		# avoid divide by zero
		if size == 0:
			continue
		score = 0.0
		# score the group based on the score for each class
		for class_val in classes:
            #al variare delle possibili classificazioni (0,1), contiamo quanti elementi del gruppo
            #(selezionato nel primo for) hanno quell'etichetta, e poi dividiamo per la cardinalità del gruppo
			p = [row[-1] for row in group].count(class_val) / size
			score += p * p
        #alla fine del for, score avrà la somma dei quadrati tra (numero di 0)/size e (numero di 1)/size
		# weight the group score by its relative size
		gini += (1.0 - score) * (size / n_instances)
        #l'indice di gini finale è calcolato considerando entrambi gli insiemi di split
	return gini

# Select the best split point for a dataset
def get_split(dataset):
	class_values = list(set(row[-1] for row in dataset))
	b_score = 2
	for index in range(len(dataset[0])-1):
		for row in dataset:
			groups = test_split(index, row[index], dataset)
			gini = gini_index(groups, class_values)
			if gini < b_score:
				b_index, b_value, b_score, b_groups = index, row[index], gini, groups
	return {'index':b_index, 'value':b_value, 'groups':b_groups, 'gini':b_score}


In [ ]:
class nodo:
    def __init__(self, nome, variabile_split=None, valore_split=None, gini=None, is_terminal=False):
        self.nome = nome
        self.variabile_split = variabile_split
        self.valore_split = valore_split
        self.gini=gini
        self.is_terminal=is_terminal
    
    def scheda_nodo(self):
        print("Scheda Nodo\n Numero:", self.nome ,"\n Variabile_split:", self.variabile_split ,"\n Valore:", self.valore_split ,"\n Gini:", self.gini ,"\n Is_terminal:", self.is_terminal,"\n")
        return        

#nodo_due = nodo(5,valore_split=5, is_terminal=True)
#print(nodo_due.scheda_nodo())

In [ ]:
def ricorsive_tree(data, max_depth,name_node=1,tree_list=[]):
    
    #check for max depth
    if max_depth==-1 and name_node!=1:
        tree_list[-1].is_terminal=True
        return tree_list
    if max_depth==0 and name_node==1:
        return tree_list
    
    
    
    split = get_split(data)
    left, right = split['groups']
    
    #creamo il nodo
    tree_list.append(nodo(name_node))
    
    # check for a no split because gini=0
    if split['gini']==0:
        tree_list[-1].is_terminal=True
    
    #aggiungiamo al nodo il criterio di split
    tree_list[-1].variabile_split = split['index']
    tree_list[-1].valore_split = split['value']
    tree_list[-1].gini = split['gini']
    
    if not tree_list[-1].is_terminal:
        #call the recursive tree, generating left and right branching
        tree_list=ricorsive_tree(left, max_depth-1,name_node=2*name_node,tree_list=tree_list)
        tree_list=ricorsive_tree(right, max_depth-1,name_node=2*name_node+1,tree_list=tree_list)
    
    return tree_list


In [ ]:

dataset = [[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]]

#call classification tree
tree = ricorsive_tree(dataset, 1)
#print_tree(tree)

In [ ]:
import numpy as np

#predictors può essere una lista oppure un array di unità statistiche


def processing_data(predictors,target):
    assert len(predictors)==len(target)
    lst=[]
    for row in predictors:
        new_row=np.append(row,target[len(lst)])
        lst.append(new_row)
    
    return np.array(lst)


In [ ]:
from sklearn.datasets import load_iris

dataset=load_iris()

dati=processing_data(dataset.data,dataset.target)

tree = ricorsive_tree(dati, 2)

In [ ]:
print("Numero totale di nodi:",len(tree),"\n")

for i in range(len(tree)):
    tree[i].scheda_nodo()

Numero totale di nodi: 5 

Scheda Nodo
 Numero: 1 
 Variabile_split: 2 
 Valore: 3.0 
 Gini: 0.3333333333333333 
 Is_terminal: False 

Scheda Nodo
 Numero: 2 
 Variabile_split: 0 
 Valore: 5.1 
 Gini: 0.0 
 Is_terminal: True 

Scheda Nodo
 Numero: 3 
 Variabile_split: 3 
 Valore: 1.8 
 Gini: 0.11030595813204513 
 Is_terminal: False 

Scheda Nodo
 Numero: 6 
 Variabile_split: 2 
 Valore: 5.0 
 Gini: 0.0856481481481482 
 Is_terminal: True 

Scheda Nodo
 Numero: 7 
 Variabile_split: 2 
 Valore: 4.9 
 Gini: 0.02898550724637681 
 Is_terminal: True 

